In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,plot_roc_curve,plot_precision_recall_curve
from sklearn.metrics import recall_score, precision_score
from imblearn.over_sampling import RandomOverSampler,SMOTE
from imblearn.pipeline import Pipeline

### Data Collection and Data Cleaning
Foi utilizada a base de dados do ano 2020 conforme solicitado pela empresa. Algumas características foram identificadas dentro da base de dados obtida: 

    (1) São muitos parâmetros presentes, sendo necessário lidar com a tarefa de escolher os principais.

    (2) Os dados são anonimizados, o que faz com que a análise fique principalmente baseada em scores de correlação, retirando a oportunidade de interpretação das features

    (3) A base é constituida de Features Contínuas, sem variáveis categóricas ou ordinais

In [2]:
#Funções de tratamento de dados

def file_correction(file):
    file.replace('na',np.nan,inplace=True)
    file[file.columns[1:]] = file[file.columns[1:]].astype('float64')
    return file

def file_encoding(file,categorical_columns=[]):
    # Target encoding
    y = LabelEncoder().fit_transform(file['class'])
    file_features = file[file.columns[1:]]
    # Features enconding
    if len(categorical_columns) > 0:
        Xcat = OneHotEncoder(drop='first').fit_transform(file_features[categorical_columns])
        # Features normalization
        non_categorical_columns = list(set(file_features.columns) - set(categorical_columns))
        Xcont = file_features[non_categorical_columns].values
        Xcont = StandardScaler().fit_transform(Xcont)
        X = np.append(Xcont,Xcat,axis=1)
    else:
        X = file_features.values
        X = StandardScaler().fit_transform(X)
    return y,X

def nulls_proportion(data):
    data = data.isna()
    not_null = data.value_counts(normalize=True)[False]
    return 1-not_null

In [3]:
df = pd.read_csv(r'./Desafio DS Dados/data_2020.csv')

### Tratamento de nulos

Identificou-se em muitas colunas dentro da base de dados. Para fazer uma limpeza na base de dados, desconsiderou-se todos aqueles em que há uma proporção > 5% de nulos em sua constituição.

In [4]:
df = file_correction(df)
less_5percent = df.apply(lambda x:nulls_proportion(x)) < 0.05
columns_toConsider = less_5percent[less_5percent==True].index
df = df[columns_toConsider]

Como a proporção de nulos é baixa em relação à base de dados e as features têm caráter de variáveis contínuas, resolveu-se preencher estes dados com a média dos valores de cada coluna, com o objetivo de ter uma base de dados mais consistente e que não será afetada pelos nulos presentes nos dados

In [5]:
df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x:x.fillna(x.mean()))

### Proporção das classes

Identificou se que se trata de um problema de classificação em que a variável target é desbalanceada contendo somente 2% da classe minoritária (pos) e o restante da classe majoritária (neg). Por meio desta constatação, observa-se que na maioria dos casos os veículos são mandados para serem consertados sem estarem com nenhum tipo de problema de ar condicionado. 

Assim, o desafio de identificar um problema no ar torna-se maior pois há uma proporção menor de dados com informações sobre a classe mais importante para o problema.

Para lidar com isto, sera usada uma técnica de resampling com o objetivo de corrigir a proporção dos dados

In [6]:
df['class'].value_counts(normalize=True).round(3)

neg    0.977
pos    0.023
Name: class, dtype: float64

### Análise de correlação entre as features e target 

Observa-se que a variável target é uma variável categórica, e por este motivo técnicas de análise de correlação/associação devem ser aplicadas considerando o seguinte cenário:

Target categórico x Feature contínua = ANOVA

Este método foi aplicando duas funções da biblioteca sklean chamadas f_classif , que utiliza a técnica ANOVA e o SelectKBest que faz o trabalho de selecionar os parâmetros de acordo com o score selecionado

In [7]:
from sklearn.feature_selection import f_classif,SelectKBest

y,X = file_encoding(df)
X = SelectKBest(f_classif,k=5).fit_transform(X, y)

C:\Users\vctor\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [74] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\vctor\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


#### Criação do modelo

Como se trata de uma base de dados desbalanceada, é necessário aplicar uma técnica de resampling. Considerando que a quantidade de dados da classe minoritária é muito pequena, escolheu-se o método de Oversample, que considera o tamanho da classe minoritária e replica a classe minoritária para ter o mesmo tamanho.

In [8]:
oversample = RandomOverSampler(sampling_strategy='minority')

#### Técnicas utilizadas

Escolheu-se técnicas com diferentes níveis de complexidade e com características distintas entre si. Assim, pode-se ter maiores critérios para a escolha do modelo, não baseando-se somente no score final.

In [ ]:
logit = LogisticRegression()
svm = SVC(probability=True)
dtc = RandomForestClassifier(max_depth=2, random_state=0)
gb = GradientBoostingClassifier()

X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train, y_train = oversample.fit_resample(X_train, y_train)

logit.fit(X_train,y_train)
svm.fit(X_train,y_train)
dtc.fit(X_train,y_train)
gb.fit(X_train,y_train)

### Avaliaçao do modelo

Utilizou-se para avaliar o modelo as métricas Precision e Recall bem como a curva ROC.

#### ROC Curve
Não foi possível fazer uma grande distinção entre os modelos pois todos tiveram bom desempenho.

In [ ]:
classifiers = [logit, svm, dtc,gb]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X, y, ax=ax)

plt.title("ROC curve comparison")
plt.legend()

#### Precision e Recall

In [ ]:
models = [LogisticRegression(), 
          RandomForestClassifier(max_depth=2, random_state=0),
          SVC(),
          GradientBoostingClassifier()]

result = list()

for model in models:
    steps = [('over', RandomOverSampler()), ('model',model )]
    pipeline = Pipeline(steps=steps)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(pipeline, X, y, scoring='precision', cv=cv, n_jobs=-1)
    result.append(scores)
    
df_scores = pd.DataFrame(np.array(result).T,
                         columns=['Logistic Regression','Random Forest','Suport Vector Machine','Gradient Boosting'])

In [ ]:
df_scores.mean().round(4)*100

In [ ]:
df_scores_plot = pd.melt(df_scores)
plt.figure(figsize=[8,5])
sns.stripplot(x='variable',y='value',data=df_scores_plot)
plt.title('Precision distribution for crossvalidation')
plt.xlabel('Model')
plt.ylabel('Recall')

In [ ]:
models = [LogisticRegression(), 
          RandomForestClassifier(max_depth=2, random_state=0),
          SVC(),
          GradientBoostingClassifier()]

result = list()

for model in models:
    steps = [('over', RandomOverSampler()), ('model',model )]
    pipeline = Pipeline(steps=steps)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(pipeline, X, y, scoring='recall', cv=cv, n_jobs=-1)
    result.append(scores)
    
df_scores = pd.DataFrame(np.array(result).T,
                         columns=['Logistic Regression','Random Forest','Suport Vector Machine','Gradient Boosting'])

In [ ]:
df_scores.mean().round(4)*100

In [ ]:
df_scores_plot = pd.melt(df_scores)
plt.figure(figsize=[8,5])
sns.stripplot(x='variable',y='value',data=df_scores_plot)
plt.title('Recall distribution for crossvalidation')
plt.xlabel('Model')
plt.ylabel('Recall')

In [ ]:
figs,axs=plt.subplots(2,2,figsize=[10,8])
count = 0
for i,model in ('Logistic Regression',logit),('Support Vector Machine',svm),('Random Forest',dtc),('Gradient Boosting',gb):
    conf_matrix = confusion_matrix(y_test,model.predict(X_test))
    sns.heatmap(conf_matrix,annot=True,ax=axs.flat[count])
    axs.flat[count].set_title(i)
    count += 1